In [ ]:
!pip install langchain-openai panel

In [ ]:
import panel
import langchain_core.messages
from langchain_openai import ChatOpenAI

panel.extension()

llm = ChatOpenAI(
    streaming=True,
    model="llama-4-scout-17b-16e-w4a16",
    temperature=0.2,
    max_retries=2,
    base_url="",
    api_key=""
)

messages = [
    langchain_core.messages.SystemMessage(content="You are a helpful assistant."),
]

async def chat_callback(contents: str, user: str, instance: panel.chat.ChatInterface):
    human_message = langchain_core.messages.HumanMessage(content=contents)
    messages.append(human_message)
    ai_message_chunks = []
    ai_message = ""
    for chunk in llm.stream(messages):
        ai_message_chunks.append(chunk.content)
        ai_message = "".join(ai_message_chunks)
        yield ai_message
    ai_final_response = ai_message_chunks[0]
    for chunk in ai_message_chunks[1:]:
        ai_final_response += chunk
    messages.append(ai_final_response)

instance = panel.chat.ChatInterface(callback=chat_callback)
instance.send("Hello, how can I help you today?", user="assistant", respond=False)
instance.servable()